In [ ]:
import torch
# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print(f"CUDA available: {cuda_available}")

if cuda_available:
    # Get the number of GPUs available
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs available: {num_gpus}")

    # Get the name of each GPU
    for i in range(num_gpus):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i}: {gpu_name}")

    # Get the current GPU memory usage
    for i in range(num_gpus):
        gpu_memory_allocated = torch.cuda.memory_allocated(i)
        gpu_memory_reserved = torch.cuda.memory_reserved(i)
        print(f"GPU {i} memory allocated: {gpu_memory_allocated / (1024 ** 3):.2f} GB")
        print(f"GPU {i} memory reserved: {gpu_memory_reserved / (1024 ** 3):.2f} GB")
else:
    print("CUDA is not available.")

In [ ]:
import sys
import os
import pickle
import logging
import numba
import napari
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
logging.getLogger('matplotlib.font_manager').disabled = True
numba_logger = logging.getLogger('numba')
numba_logger.setLevel(logging.WARNING)

matplotlib_logger = logging.getLogger('matplotlib')
matplotlib_logger.setLevel(logging.WARNING)

# Add the src directory to sys.path
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(src_path)

from src.IndependentSteps import Pycromanager2H5, FFF2H5, DownloadData, Pycro

from src.SequentialSteps import BIGFISH_SpotDetection, SimpleCellposeSegmentaion, CellProperties, DilationedCytoMask ,\
                                    DisplaySequentialParams, IlluminationCorrection, Calculate_BIGFISH_Threshold, rescale_images,\
                                    Automatic_BIGFISH_Threshold

from src.FinalizationSteps import Save_Outputs, Save_Images, Save_Parameters, Save_Masks, return_to_NAS, remove_local_data_but_keep_h5, save_copy, remove_local_data

from src.Parameters import Parameters, Experiment, Settings, ScopeClass, DataContainer

from src.Displays import Display

from src.GUI import GUI, StepGUI

from src.Pipeline import Pipeline

In [ ]:
# This is how you initialize the parameters
scope = ScopeClass() 
data = DataContainer()
settings = Settings(name='ANALYSIS_NAME')
experiment = Experiment()

print(experiment)
print(settings)
print(scope)
print(data)

In [ ]:
experiment.initial_data_location = ['NAS_LOCATION']

data.local_dataset_location = None

experiment.FISHChannel = FISH_CHANNEL
experiment.nucChannel = NUC_CHANNEL
experiment.cytoChannel = CYTO_CHANNEL
experiment.voxel_size_z = 500

settings.num_chunks_to_run = 1

Parameters().get_parameters()
Parameters.validate()

In [ ]:
settings.load_in_mask = True

DownloadData().run()
Pycro().run()

In [ ]:
settings.cellpose_min_size = 500
settings.cellpose_diameter = [180, 90]
settings.cellpose_pretrained_model = ["GAPDH_cyto", 'DAPI_nuclei'] 

SimpleCellposeSegmentaion().run()

In [ ]:
scope.spot_yx = 130
scope.spot_z = 360
scope.voxel_size_yx = 100

BIGFISH_SpotDetection().run()

In [ ]:
viewer = napari.Viewer()
viewer.add_image(data.images, channel_axis=2, scale=(5,1,1))

# Extract the relevant columns from the dataframe
spot_results = data.spotresults
z_coords = spot_results['z_px'].values
y_coords = spot_results['y_px'].values
x_coords = spot_results['x_px'].values
fov_coords = spot_results['fov'].values
time_coords = spot_results['timepoint'].values

# Prepare the points for napari
points = []
for z, y, x, fov, time in zip(z_coords, y_coords, x_coords, fov_coords, time_coords):
    points.append([fov, time, 0, z, y, x])  # Assuming channel is 0 for visualization

points = np.array(points)

# Add the points to the napari viewer
viewer.add_points(
    points,
    size=1,  # Adjust the size of the points
    face_color='red',
    edge_color='red',
    ndim=6,  # Match the dimensionality of the image
    name='Spot Results',
    scale=(5,1,1)
)

In [ ]:
CellProperties().run()

In [ ]:
Save_Outputs().run()
Save_Parameters().run()
Save_Images().run()
return_to_NAS()

settings.data_names = []
settings.copy_locations = []
save_copy().run()

remove_local_data().run()

In [ ]:
all_datasets = ['ALL_NAS_LOCATIONS']
experiment.initial_data_location = all_datasets

pipeline = Pipeline()
pipeline.run_on_cluster('CLUSTER_PATH_ON_CLUSTER', name=settings.name)